# 1 Preparing

In [966]:
%pip install pymystem3

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [967]:
%pip install xlrd

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [968]:
%pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [969]:
%pip install pymorphy2

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [970]:
import pandas as pd
import regex as rg
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

import string
from pymystem3 import Mystem

import pickle

import pymorphy2

import os
from pathlib import Path

In [971]:
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
# from nltk.corpus import stopwords

# STOPWORDS = set(stopwords.words("russian"))

# 2 Load data and prepare  setrs

In [988]:
class_file = {}
dictionary_files = {}

# *** root directory ***
pwd = '/home/jupyter/work/resources'

# Target dataset (which is used for the true classification)
key_value = "coffee.xlsx"
dict_name = 'coffee_dict.xlsx'

# *** Datasets ***
dataset_path = pwd + '/content/datasets/'
for entry in os.listdir(dataset_path):
    if os.path.isfile(os.path.join(dataset_path, entry)):
        class_file[entry] = dataset_path + entry

# *** Dictionaries with brands and support words ***
# PLS be shure that name of the support dectionary file is equal with "dict_name"
dataset_path = pwd + '/content/dictionaries/'
for entry in os.listdir(dataset_path):
    if os.path.isfile(os.path.join(dataset_path, entry)):
        dictionary_files[entry] = dataset_path + entry

# used for datasets fields
# req_str - text of requests
# cls_str - binary classification, 1 - Trues, 0 - False
req_str = 'text'
cls_str = 'group'

# The structure of the dictionary fiels, and filds which are drpoped during operations
dictionary_fields = ['brand_id', 'brand', req_str]
dictionary_dropped = ['brand_id', 'brand']

# The frequency (weight) for brand words (uses for weight increasing of the brands, brand translations, etc...)
weight_multiplicator = 32

In [974]:
# Define the positive dataset
print("*** Define the positive dataset ***")

dataset_raw = pd.read_excel(class_file[key_value], header=None, names=[req_str], engine='openpyxl')
dataset_raw[cls_str] = 1
print("Positive Dataset")
print(dataset_raw.head(-4))
print("Positive Dataset shape: ", dataset_raw.shape)

*** Define the positive dataset ***
Positive Dataset
                               text  group
0              кофе новогодняя ночь      1
1               кофе мягкой обжарки      1
2                     армпромсервис      1
3                   кофе белый арап      1
4     кофе в зернах бразилия купить      1
...                             ...    ...
7446        где купить молотый кофе      1
7447       кофе растворимый в пачке      1
7448                 кофе palombini      1
7449              кубита торрефакто      1
7450     кофе в зернах монтана кофе      1

[7451 rows x 2 columns]
Positive Dataset shape:  (7455, 2)


In [975]:
# Define the negative dataset
print("*** Define the negative dataset ***")

cont_dataset = pd.DataFrame()

for key, value in class_file.items():
    if key != key_value:
        tmp = pd.read_excel(value, header=None, names=[req_str], engine='openpyxl')
        tmp[cls_str] = 0
        
        print(f"Size of {key}:", tmp.shape)
        
        # Checking that your target dataset does not contain the fields from other sets
        dataset_raw.drop(dataset_raw[dataset_raw[req_str].isin(tmp[req_str])].index, inplace=True)
        print(f"Positive dataset shape {dataset_raw.shape}, after duplication cleaning in {key_value} by fields from {key}")
        
        # Add a new part to the negative dataset 
        cont_dataset = pd.concat([cont_dataset, tmp], sort=False)

# Shafle cont_dataset
cont_dataset = cont_dataset.sample(frac = 1).reset_index(drop=True)
        
print(f"Positive dataset shape {dataset_raw.shape}, after dublication cleaning in {key_value}")
print("Negative set:")
print(cont_dataset)
print("Size of the negative set:", cont_dataset.shape)

*** Define the negative dataset ***
Size of zoo.xlsx: (69267, 2)
Positive dataset shape (7455, 2), after duplication cleaning in coffee.xlsx by fields from zoo.xlsx
Size of pampers.xlsx: (49175, 2)
Positive dataset shape (7455, 2), after duplication cleaning in coffee.xlsx by fields from pampers.xlsx
Size of toys.xlsx: (451274, 2)
Positive dataset shape (7455, 2), after duplication cleaning in coffee.xlsx by fields from toys.xlsx
Positive dataset shape (7455, 2), after dublication cleaning in coffee.xlsx
Negative set:
                                        text  group
0                            собака на заказ      0
1       купить подгузники хаггис элит софт 3      0
2                      боевая голова игрушка      0
3                        игрушка медведь 100      0
4                            всё лего оружие      0
...                                      ...    ...
569711               ковер детский малиновый      0
569712                          купить вулик      0
569713  

In [989]:
# 4 
# Let us define the train data set which we are going to use during our model fitting, 
# and the validation dataset for the following checking of the model.

print("*** Define train and validation datasetes ***")

# Train Dataset
df_train = pd.concat([
    dataset_raw[:(round(dataset_raw.shape[0]*0.75))],
    cont_dataset[:(round(cont_dataset.shape[0]*0.75))]
]).sample(frac = 1).reset_index(drop=True)

# Validation dataset
df_val = pd.concat([
    dataset_raw[(round(dataset_raw.shape[0]*0.75)):],
    cont_dataset[(round(cont_dataset.shape[0]*0.75)):]
]).sample(frac = 1).reset_index(drop=True)

print("Sizes of our dataset before operations: ", dataset_raw.shape, cont_dataset.shape)
print("The dataset size for the model fitting: ", df_train.shape)
print("The dataset size for the model checking: ", df_val.shape)

*** Define train and validation datasetes ***
Sizes of our dataset before operations:  (7455, 2) (569716, 2)
The dataset size for the model fitting:  (432878, 2)
The dataset size for the model checking:  (144293, 2)


In [990]:
# 5 
# Adding words from support dictionary 
# PLS check that the name of the support dictionary file is equal with "dict_name" value

print("*** Adding words from support dictionary ***")
dictionary = pd.DataFrame(columns=dictionary_fields)

for d_key, d_value in dictionary_files.items():
    if d_key == dict_name:
        tmp = pd.read_excel(d_value, names=dictionary_fields, engine='openpyxl')
        tmp[cls_str] = 1
        dictionary = dictionary.append(tmp, sort=False)

dictionary = dictionary.drop(dictionary_dropped, axis = 1)

print("Dictionary dataset shape:", dictionary.shape)
print(dictionary.head(-4))

print("Train dataset shape before dictionary adding:", df_train.shape)

for _ in range(weight_multiplicator):
    df_train = pd.concat([df_train, dictionary], sort=False)

df_train  = df_train.reset_index(drop=True)

print("Train dataset shape after dictionary adding:", df_train.shape)

*** Adding words from support dictionary ***
Dictionary dataset shape: (1966, 2)
               text  group
0            вошеба    1.0
1            Восеба    1.0
2            Woseba    1.0
3            восеба    1.0
4            woseba    1.0
...             ...    ...
1957  нэ скафе голд    1.0
1958    нескофеголд    1.0
1959  нескафе  голд    1.0
1960  не скафе голд    1.0
1961        bourbon    1.0

[1962 rows x 2 columns]
Train dataset shape before dictionary adding: (432878, 2)
Train dataset shape after dictionary adding: (495790, 2)


In [991]:
# 6 Prepare X and Y for the model fitting
print("*** Prepare X and Y ***")

# Train
x_train = df_train.drop([cls_str], axis = 1)
y_train = df_train[cls_str]
print("Test X, Y shapes: ", x_train.shape, y_train.shape)

# Validation
x_val = df_val.drop([cls_str], axis = 1)
y_val = df_val[cls_str]

print("Validation X, Y shapes: ", x_val.shape, y_val.shape)

*** Prepare X and Y ***
Test X, Y shapes:  (495790, 1) (495790,)
Validation X, Y shapes:  (144293, 1) (144293,)


In [992]:
# 7 Prepare the Pipeline, vectoiriser and SGD classifier 
print("*** Prepare the Pipeline, vectoiriser and SGD classifier ***")

sgd_ppl_clf = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
    ('sgd_clf', SGDClassifier(
        random_state=None,
        class_weight=None,
        loss='perceptron', 
        penalty='l2'
    ))])

print(sgd_ppl_clf)

*** Prepare the Pipeline, vectoiriser and SGD classifier ***
Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                 SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                    

In [993]:
print("*** Fit SGD classifier  ***")
sgd_ppl_clf.fit(x_train[req_str], y_train)

*** Fit SGD classifier  ***


Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                 SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='perceptron',
      

# 2 Check the model

In [994]:
print("*** Check the model by the validation data ***")

predicted_sgd = sgd_ppl_clf.predict(x_val[req_str])
print(metrics.classification_report(predicted_sgd, y_val))

*** Check the model by the validation data ***
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    142476
         1.0       0.96      0.98      0.97      1817

    accuracy                           1.00    144293
   macro avg       0.98      0.99      0.99    144293
weighted avg       1.00      1.00      1.00    144293



In [995]:
print("*** Check and calculate the metrics ourselves ***")

# model prediction result
a_check = predicted_sgd.view()
# Validation resuls  
b_check = np.array(y_val)
# проверим размерность
print("Check shapes of the prediction and Y validation:", a_check.shape, b_check.shape)

# True Negative Predicted - false, Real - false
ab_diff_tn = []
# False Negative Predicted - false, Real - True
ab_diff_fn = []
# False positive Predicted - True, Real - False
ab_diff_fp = []
# True positive Predicted - True, Real - True
ab_diff_tp = []

print('')
print('******')
print(key_value)

for indx in range(len(a_check)):
    if (a_check[indx] == True):
        if (b_check[indx] == True):
            ab_diff_tp.append(indx)
        else:
            ab_diff_fp.append(indx)
    else:
        if (b_check[indx] == True):
            ab_diff_fn.append(indx)
        else:
            ab_diff_tn.append(indx)
            
tn =  len(ab_diff_tn)
fn =  len(ab_diff_fn)
fp =  len(ab_diff_fp)
tp =  len(ab_diff_tp)

print(f"True Negative:  {tn}  | False Positive:  {fp}")
print(f"False Negative: {fn}   | True Positive:  {tp}")

print(f"Precision = {tp} / ( {tp} + {fp} ) = {round(tp/(tp+fp), 2)}")
print(f"Recall = {tp} / ( {tp} + {fn} ) = {round(tp/(tp+fn), 2) }")

print('')
print('******')
print(f"Accurracy = {round(metrics.accuracy_score(a_check, b_check), 2) }")
print(f"F1 Score = {round(metrics.f1_score(a_check, b_check, average='weighted'), 2) }")

*** Check and calculate the metrics ourselves ***
Check shapes of the prediction and Y validation: (144293,) (144293,)

******
coffee.xlsx
True Negative:  142398  | False Positive:  31
False Negative: 78   | True Positive:  1786
Precision = 1786 / ( 1786 + 31 ) = 0.98
Recall = 1786 / ( 1786 + 78 ) = 0.96

******
Accurracy = 1.0
F1 Score = 1.0


# 3 Stream classification

In [996]:
df_stream = pd.read_csv(pwd + '/content/stream/Поток_30К_уникальные.csv')
df_stream["QueryText"] = df_stream["QueryText"].str.replace('\d+', '')
df_stream = df_stream.fillna('')
df_stream.head(2)

,QueryText
0,стяжки fortisflex
1,каэдэ косплей


In [997]:
df = pd.DataFrame()

predicted_stream = sgd_ppl_clf.predict(df_stream["QueryText"])

df["Request"] = df_stream["QueryText"]
df["Predicted"] = predicted_stream
df2 = df[df["Predicted"]==True]
df2.to_csv(pwd + "/content/ab_analyse/coffee_predictions.csv")
df2

,Request,Predicted
26,Unlimited Nothing Like The Rain,1.0
51,соляной шоколадно-кофейный скраб для тела массаж,1.0
162,диара капсулы цена,1.0
322,как сделать фокус с колодой карт,1.0
332,письмо Минэкономразвития России от .. № Ди-,1.0
...,...,...
29581,black desert фотофильтр,1.0
29620,Grand+Teft+Auto+IV,1.0
29628,Артозакс в капсулах АлтайСашемед,1.0
29667,первородный гадальная карта,1.0
